# Survival Analysis: K-M Plot with Confidence Band
## Objective and Context
- The paper aims to enhance Kaplan-Meier (K-M) plots using SAS options and ODS template modifications, focusing on PROC LIFETEST to overcome limitations in survival analysis plotting.
## Enhancement Method
- It involves using ODS templates with PROC LIFETEST for creating varied K-M plots, adding statistics, and visual customizations.
## Customization Details
- Guides on modifying ODS templates to customize plot elements like titles, labels, axes, legends, and inset tables.

## Code Block 1: Basic Survival Plot with Confidence Bands
```sas
ODS GRAPHICS ON;
Proc lifetest data=data1 method=KM plots=survival(cb);
time Time*Censor(0);
strata Treatment; 
run; 
ODS GRAPHICS OFF;
```
![image1](1.png)
- ODS GRAPHICS ON/OFF: Toggles the graphical output feature in SAS.
- Proc lifetest: Invokes the LIFETEST procedure for survival analysis.
- data=data1: Specifies the dataset 'data1'.
- method=KM: Uses the Kaplan-Meier method for survival analysis.
- plots=survival(cb): Requests a survival plot with confidence bands.
- time Time*Censor(0): Defines the time-to-event and censoring variables.
- strata Treatment: Stratifies the analysis based on the 'Treatment' variable.


## Code Block 2: Enhanced Plot with At-Risk and Test Options
```sas
ODS GRAPHICS ON;
Proc lifetest data=data1 method=KM plots=survival(cb ATRISK TEST);
time Time*Censor(0);
strata Treatment / test=logrank; 
run; 
ODS GRAPHICS OFF;
```
![image2](2.png)
- plots=survival(cb ATRISK TEST): Requests a survival plot with confidence bands, at-risk information, and test statistics.
- test=logrank: Applies the log-rank test for comparing survival curves.

## Code Block 3: Template Modification for Customized Statistics
```sas
proc template; 
define statgraph Stat.Lifetest.Graphics.ProductLimitSurvival; 
BeginGraph; 
layout overlay / xaxisopts=(label="Days") yaxisopts=(label="Survival Probability");
dynamic StrVal1 NObs1 NEvent1 Median1 LowerMedian1 UpperMedian1 
Strval2 NObs2 NEvent2 Median2 LowerMedian2 UpperMedian2; 
layout gridded/columns=3 border=TRUE autoalign=(TOPRIGHT BOTTOMLEFT TOP BOTTOM);
entry " "; entry halign=center strval1; entry halign=center strval1; 
entry halign=left "No. of Patients"; entry halign=center NObs1; entry halign=center NObs1; 
entry halign=left "Event"; entry halign=center NEvent1; entry halign=center NEvent1; 
entry halign=left "Censor"; entry halign=center eval(NObs1 - NEvent1);
entry halign=center eval(NObs1 - NEvent1);
entry halign=left "Median"; entry halign=center eval(strip(put(Median15.1)));
entry halign=center eval(strip(put(Median25.1)));
entry halign=left "Median CI";
entry halign=center "(" eval(strip(put(lowermedian15.1))) "-" eval(strip(put(uppermedian15.1))) ")";
entry halign=center “(" eval(strip(put(lowermedian25.1))) "-" eval(strip(put(uppermedian25.1))) ")";
endlayout; 
EndGraph; 
end;
```
![image3](3.png)
### Specific Code Component: define statgraph Stat.Lifetest.Graphics.ProductLimitSurvival;
- Purpose: This line starts the definition of a new graph template specific to the Kaplan-Meier survival plot. It signifies that the subsequent modifications and customizations will be applied to this particular type of survival plot.
- Usage in Paper: Used in code blocks where the authors demonstrate template modifications for customizing the survival plot's appearance and the display of statistical information.
### Specific Code Component: BeginGraph;
- Purpose: This statement indicates the beginning of the graph definition within the template. It's where the customization and layout specifications of the graph start.
- Usage in Paper: Marks the start of each graph customization segment, leading into detailed layout and styling specifications.
### Specific Code Component: layout overlay;
- Purpose: The layout overlay statement is used to create layered graphical elements within the plot. It allows the combination of various plot types (like step plots, band plots, scatter plots) in a single graph area, making it possible to overlay different types of data representations.
- Usage in Paper: This layout is crucial for adding multiple statistical elements and visual layers to the Kaplan-Meier plot, enabling the overlay of statistical data, confidence bands, and other graphical elements onto the survival plot.

- dynamic ...: Declares dynamic variables to hold statistics for each stratum. The variables StrVal1, NObs1, NEvent1, Median1, LowerMedian1, UpperMedian1, Strval2, NObs2, NEvent2, Median2, LowerMedian2, and UpperMedian2 represent stratum labels, number of observations, number of events, median survival times, and lower and upper bounds of median survival times for two strata.
- The layout then uses these dynamic variables to display these statistics on the plot. This adds a layer of detailed statistical information directly onto the Kaplan-Meier plot, enhancing its informational value.




## Code Block 4: Graph Title and Axis Label Modification
```sas
proc template; 
define statgraph Stat.Lifetest.Graphics.ProductLimitSurvival; 
BeginGraph; 
entrytitle "MAIN TITLE GOES HERE ";
if (EXISTS(SECONDTITLE))
  entrytitle "SECOND TITLE GOES HERE "
endif; 
layout overlay / xaxisopts=(label="X-AXIS Label" shortlabel=XNAME offsetmin=.05 
linearopts=(viewmax=MAXTIME tickvaluelist=(0 5 10 15 20 25)))
yaxisopts=(label="Y-AXIS Label" shortlabel="Survival" linearopts= (viewmin=0
viewmax=1 tickvaluelist=(0 0.2 0.4 0.6 0.8 1.0)));
...
```
![image4](4.png)
- This code customizes the main and secondary titles, and the X and Y-axis labels and options. The entrytitle statements set the titles, while xaxisopts and yaxisopts adjust axis labels and tick marks.

## Controlling the Location of Inset Tables

```sas
proc template;
 define statgraph Stat.Lifetest.Graphics.ProductLimitSurvival;
 dynamic NStrata xName plotAtRisk plotCensored plotCL plotHW plotEP labelCL labelHW labelEP
 maxTime method StratumID classAtRisk plotBand plotTest GroupName yMin Transparency
 SecondTitle TestName pValue;
 BeginGraph;
 entrytitle "MAIN TITLE GOES HERE ";
 entrytitle "SECOND TITLE GOES HERE ";
 layout lattice/rows=2 columns=1 columndatarange=unionall rowweights=(.90 .10);
 layout overlay / xaxisopts=(label="X-Aaxis Label" offsetmin=.05 linearopts= (tickvaluelist=(0 5 10 15
 20 25)))
 yaxisopts=(label="Y-Aaxis Label" linearopts=(tickvaluelist=(0 .2 .4 .6 .8 1.0)));
 if (PLOTHW)
 bandplot LimitUpper=HW_UCL LimitLower=HW_LCL x=TIME / group=STRATUM
 index=STRATUMNUM modelname="Survival" datatransparency=Transparency;
 endif;
 stepplot y=SURVIVAL x=TIME / group=STRATUM index=STRATUMNUM name="Survival"
 rolename=(_tip1=ATRISK _tip2=EVENT) tip=(y x Time _tip1 _tip2);
 if (PLOTCENSORED)
 scatterplot y=CENSORED x=TIME / group=STRATUM index=STRATUMNUM arkerattrs=
 (symbol=plus);
 endif;
 Opaque=true;
dynamic TestName pValue NObs1 NEvent1 Median1 strval1 NObs2 NEvent2 Median2 strval2;
 layout gridded/columns=3 border=TRUE autoalign=(TopRight);
 entry ""; entry halign=center strval1 ; entry halign=center strval2;
 entry halign=left "No. of Patients"; entry halign=center NObs1; entry halign=center NObs2;
 entry halign=left "Event"; entry halign=center NEvent1; entry halign=center NEvent2;
 entry halign=left "Censor";entry halign=center eval(NObs1 - NEvent1); entry halign=center
eval(NObs2 - NEvent2);
entry halign=center TESTNAME " P-Value=" eval (PUT(PVALUE, PVALUE6.4));
endlayout;
endlayout;
layout overlay / xaxisopts=(display=none);
 blockplot x=TATRISK block=ATRISK / class=CLASSATRISK
 repeatedvalues=true display=(label values) valuehalign=start valuefitpolicy=truncate
 labelposition=left labelattrs=GRAPHVALUETEXT includemissingclass=false;
 endlayout;
 endlayout;
 EndGraph;
 end;
 run;
```

- define statgraph Stat.Lifetest.Graphics.ProductLimitSurvival;

Defines a new statistical graph template specifically for the Kaplan-Meier survival plot.

- dynamic ...;

Declares dynamic variables used in the graph. These variables (NStrata, xName, plotAtRisk, etc.) allow for flexible input and customization within the graph.

- BeginGraph;

Marks the beginning of the graph's layout definition.

- entrytitle "MAIN TITLE GOES HERE ";

Sets the main title of the graph.

- entrytitle "SECOND TITLE GOES HERE ";

Sets a secondary title, if needed.

- layout lattice/rows=2 columns=1 columndatarange=unionall rowweights=(.90 .10);

Establishes a lattice layout with two rows and one column, allocating 90% of the space to the top row (main plot) and 10% to the bottom row (for inset tables or additional information).

- layout overlay / xaxisopts=(label="X-Aaxis Label" offsetmin=.05 linearopts= (tickvaluelist=(0 5 10 15 20 25)))

Within the lattice, creates an overlay layout for the top row. Customizes the X-axis options including label, offset, and tick values.

- yaxisopts=(label="Y-Aaxis Label" linearopts=(tickvaluelist=(0 .2 .4 .6 .8 1.0)));

Customizes the Y-axis options similar to the X-axis, setting labels and tick values.

- if (PLOTHW) bandplot LimitUpper=HW_UCL LimitLower=HW_LCL x=TIME / group=STRATUM index=STRATUMNUM modelname="Survival" datatransparency=Transparency; endif;

Conditional statement to add a band plot (Hall-Wellner confidence bands) if specified by PLOTHW. This plots upper and lower confidence limits (HW_UCL, HW_LCL) against TIME.

- stepplot y=SURVIVAL x=TIME / group=STRATUM index=STRATUMNUM name="Survival" rolename=(_tip1=ATRISK _tip2=EVENT) tip=(y x Time _tip1 _tip2);

Creates a step plot of survival over time. Groups data by STRATUM and includes tool tips (_tip1, _tip2) for additional information display.

- if (PLOTCENSORED) scatterplot y=CENSORED x=TIME / group=STRATUM index=STRATUMNUM markerattrs=(symbol=plus); endif;

Conditional statement to add a scatter plot for censored data points if PLOTCENSORED is true.


- dynamic TestName pValue NObs1 NEvent1 Median1 strval1 NObs2 NEvent2 Median2 strval2;

Additional dynamic variables for displaying test names, p-values, number of observations, events, and median values for two strata.

- layout gridded/columns=3 border=TRUE autoalign=(TopRight);

Defines a grid layout with 3 columns for the bottom row of the lattice, used for displaying additional statistics or information aligned to the top right.

- entry ...;

These statements populate the gridded layout with specific statistics and information, such as number of patients, events, and median survival times.

- blockplot x=TATRISK block=ATRISK / class=CLASSATRISK ...;

A block plot used for showing at-risk information over time. The class=CLASSATRISK option allows categorization of the at-risk data.

- EndGraph; 

concludes the graph definition. end; closes the template definition, and run; executes the PROC TEMPLATE code.

![image5](5.png)

## Conclusions
The paper concludes that with SAS 9.2 and later versions, users have extensive capabilities to customize survival plots. By modifying elements like titles, labels, axes, and inset tables using ODS Statistical Graphics, more informative and visually appealing survival analysis results can be achieved. The specific use of define statgraph, BeginGraph, and layout overlay in template customization plays a key role in enhancing the visual and informational quality of Kaplan-Meier plots.